In [1]:
#! pip install py2neo;
#!pip install neo4j
#from py2neo import Graph
from neo4j import GraphDatabase
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import numpy as np


In [4]:
#graph = Graph("bolt://3.220.233.169:7687", auth=("neo4j", "i-0e23d19f0d8795714"))

In [29]:
uri = "bolt://3.220.233.169:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "i-0e23d19f0d8795714"))

This is coding through one example prediction piece by piece so that I can refactor this code into prediction functions for arbitrary person and business

id:195103, Chong. carlucciios tivoli gardens, id:17


First phase is user preference. First step is getting number of reviews, average review score, and review score distribution

In [66]:
review_dist=graph.run("""
MATCH (u:User)-[:WROTE]->(r:Review)
WHERE ID(u)=195103
RETURN ID(r), r.stars
""").to_data_frame()

In [67]:
review_stars=review_dist['r.stars'].value_counts()

In [68]:
review_stars

4.0    18
5.0    15
3.0    13
2.0     3
Name: r.stars, dtype: int64

In [129]:
mean_review=review_dist['r.stars'].mean()
mean_review

3.9183673469387754

In [70]:
num_reviews=review_dist['r.stars'].shape[0]

In [71]:
tempdf=graph.run("""
MATCH (b:Business)-[:IN_CATEGORY]->(c:Category)
WHERE ID(b)=17
RETURN ID(c), c.id
""").to_data_frame()

In [72]:
tempdf

,ID(c),c.id
0,6003,Restaurants
1,6052,Italian


In [73]:
cat_id=list(tempdf['ID(c)'])

In [74]:
cat_id

[6003, 6052]

In [75]:
bizlist=[]
for cat in cat_id:
    temp=graph.run(f"MATCH (b:Business)-[:IN_CATEGORY]->(c:Category)\
    WHERE ID(c)={cat} RETURN ID(b)").to_series()
    bizlist.append(temp)

In [103]:
list(bizlist[1])[0:2]

[191757, 92280]

In [77]:
similar_biz=[]
for i in range(len(bizlist)):
    sim_biz = graph.run(f"MATCH (u:User)-[:WROTE]->(r:Review)-[:REVIEWS]->(b:Business) WHERE ID(u)=195103 and ID(b) IN {list(bizlist[i])}\
    RETURN r.stars, ID(b)").to_data_frame()
    similar_biz.append(sim_biz)


In [78]:
similar_biz

[     ID(b)  r.stars
 0    18179      3.0
 1    20633      5.0
 2    15420      5.0
 3    20728      4.0
 4    10924      4.0
 5    33784      2.0
 6    24002      3.0
 7    33755      4.0
 8    49324      3.0
 9    57019      3.0
 10   62704      5.0
 11   91540      5.0
 12   84569      4.0
 13   91118      5.0
 14  103683      5.0
 15  114253      4.0
 16   99165      3.0
 17  113211      5.0
 18  110749      4.0
 19  122389      4.0
 20  149120      4.0
 21  144251      4.0
 22  153199      3.0
 23  151133      2.0
 24  167760      4.0
 25  159921      3.0
 26  188708      5.0,     ID(b)  r.stars
 0   62704      5.0
 1  113211      5.0]

In [79]:
similar_biz[0]['r.stars'].value_counts()

4.0    10
5.0     8
3.0     7
2.0     2
Name: r.stars, dtype: int64

In [80]:
similar_biz[1]['r.stars'].value_counts()

5.0    2
Name: r.stars, dtype: int64

In [92]:
numerator=np.empty(5)
for i in (1,2,3,4,5):
    try:
        numerator[i-1]=review_stars[i]
    except:
        numerator[i-1]=0


In [97]:
PRu=(numerator+1)/(num_reviews+5)

In [119]:
num_cat=len(similar_biz)
cats_by_stars=np.empty((num_cat,5))

for i in range(num_cat):
    cat_stars=similar_biz[i]['r.stars'].value_counts()
    for j in (1,2,3,4,5):
        try:
            cats_by_stars[i][j-1]=cat_stars[j]
        except:
            cats_by_stars[i][j-1]=0

cats_by_stars

array([[ 0.,  2.,  7., 10.,  8.],
       [ 0.,  0.,  0.,  0.,  2.]])

In [122]:
PRaj=( (cats_by_stars+1) / (numerator+num_cat) ).prod(axis=0)


array([0.25      , 0.12      , 0.03555556, 0.0275    , 0.09342561])

In [126]:
user_prefs_un_normalized=PRu*PRaj

In [127]:
user_prefs=user_prefs_un_normalized/sum(user_prefs_un_normalized)

In [128]:
user_prefs

array([0.07703953, 0.14791589, 0.15339426, 0.16101261, 0.46063772])

In [ ]:
with driver.session() as session:
    result = session.run("\
MATCH (u:User) \
UNWIND split(u.friends,',') as friendlist \
RETURN DISTINCT friendlist")

In [6]:
friends=graph.run("\
MATCH (u:User) \
UNWIND split(u.friends,',') as friendlist \
RETURN DISTINCT friendlist").to_series()

In [7]:
friends.shape

(14906537,)

In [ ]:
sum(friends.duplicated()

In [ ]:
users=graph.run("\
MATCH (u:User) \
RETURN u.id").to_series()

In [14]:
with driver.session() as session:
    result = session.run("\
MATCH (u:User) \
RETURN u.id")


In [15]:
users=result.values()

In [18]:
df = pd.DataFrame(users, columns =['userid'])

In [25]:
with driver.session() as session:
    result = session.run("\
MATCH (b:Business) \
RETURN distinct b.city")

In [26]:
cities=result.values()


In [66]:
cities_df = pd.DataFrame(cities, columns =['city'])

In [67]:
cities_df.shape

(1204, 1)

In [68]:
# for city in cities_df['userid']
# with driver.session() as session:
#     result = session.run("\
# MATCH (b:Business) \
# RETURN distinct b.city")

In [69]:
cities_df2=pd.DataFrame()

In [70]:
cities_df2['city']=cities_df['city'].str.strip()
cities_df2['city']=cities_df['city'].str.lower()

In [71]:
cities_df2=cities_df2.drop_duplicates()

In [73]:
cities_df2.shape

(1095, 1)

In [74]:
cities_df2.head()

,city
0,phoenix
1,mississauga
2,charlotte
3,goodyear
4,calgary


In [92]:
# for city in cities_df2['city']:
#     with driver.session() as session:
#         result = session.run(f'\
#     CREATE (c:City) \
#     SET c.name="{city}" ')

In [85]:
summ=result.summary()

'    MERGE (c:City)     SET c.name="spring hill city view" '

In [98]:
with driver.session() as session:
    result = session.run('\
    MATCH (b:Business) \
    MATCH (c:City) WHERE c.name=tolower(trim(b.city))\
    SET c += {state:b.state}\
    CREATE (b)-[:IN_CITY]->(c)')
   


KeyboardInterrupt: 

In [124]:
with driver.session() as session:
    result = session.run('\
    MATCH (b:Business) \
    RETURN apoc.node.degree(b,"IN_CITY")')

In [125]:
bizdegree=result.values()


In [126]:
bizdegdf=pd.DataFrame(bizdegree)

In [127]:
bizdegdf[0].value_counts()

1    192609
Name: 0, dtype: int64

Fixed two of them manuaslly, created one edge deleted one edge

In [176]:
with driver.session() as session:
    result = session.run('\
    MATCH (c:City) \
    RETURN distinct c.state')

In [177]:
states=pd.DataFrame(result.values(),columns=['state'])

In [178]:
states

,state
0,AZ
1,CA
2,QC
3,NV
4,OH
5,WI
6,AB
7,UT
8,PA
9,NC


In [148]:
with driver.session() as session:
    result = session.run('\
    MATCH (c:City) \
    MERGE (s:State {name:c.state}) \
    ON CREATE SET s.name=c.state \
    MERGE (c)-[:IN_STATE]->(s)')

In [163]:
with driver.session() as session:
    result = session.run('\
    MATCH (s:State)<-[:IN_STATE]-(c:City)\
    RETURN s.name, apoc.node.degree(s,"IN_STATE"), c.name, apoc.node.degree(c,"IN_CITY")')

In [164]:
counts_by_state=result.values()

In [165]:
countsdf=pd.DataFrame(counts_by_state)

In [170]:
countsdf.groupby(0).sum()

,1,3
0,,
AB,676,8008
AL,1,1
AZ,10404,56691
CA,121,12
CON,1,1
CT,1,1
DOW,1,1
FL,1,1
IL,729,1933


In [13]:
with driver.session() as session:
    result = session.run('\
    MATCH (s:State)<-[:IN_STATE]-(c:City)<-[:IN_CITY]-(b:Business)\
    RETURN  b.name, apoc.node.degree(b,"REVIEWS"), s.name, c.name')

In [14]:
rev_counts=result.values()

In [17]:
rev_countsdf=pd.DataFrame(rev_counts)

In [20]:
rev_countsdf.head()

,0,1,2,3
0,Homestead Steaks,8,NE,omaha
1,"Tiffani Tonkinson, DDS",5,AL,henderson nevada
2,Di Dee's Diner,7,SC,lancaster
3,Redbarn at Larkspur Ranch,3,SC,lancaster
4,Spa & Bodywork,5,SC,lancaster


In [21]:
rev_counts_by_state=rev_countsdf.groupby(2).sum()

In [22]:
rev_counts_by_state

,1
2,
AB,99616
AL,5
AZ,2083011
CA,179
CON,3
CT,10
DOW,4
FL,4
IL,42378


alright cities and states are implemented

In [5]:
def cypher(driver, query, results_columns):
    """input is a neo4j connection driver, a string representing a cypher query
    and a list of string for data frame column names. returns the dataframe of the results."""
    
    with driver.session() as session:
        result = session.run(query)
        
    result_df = pd.DataFrame(result.values() , columns=results_columns)
    
    return result_df

In [ ]:
cypher(driver,"MATCH (u:User)-[:WROTE]->(:Review)-[:REVIEWS]->(:Business)-[:IN_CITY]->(:City)-[:IN_STATE]->(s:State)\
CREATE (u)-[:REVIEWS_IN]->(s)", None)
